In [46]:
#==============================================================================================================================
#==============================================================================================================================
                                                #Author: Vatsal Gaurang Shah
                                                #Description: Pre-processing data for BusesRoutes
#==============================================================================================================================
#==============================================================================================================================
import pandas as pd

In [47]:
sheet_name = '376  Rout name, Stage & LL'
file_path = '/Users/vatsalshah/Documents/Workspace/SEM2/DV Project/BRT  Non BRT Route Details  Bus Stop LatLong (1).xls'

data = pd.read_excel(file_path, sheet_name=sheet_name)
filtered_columns = [col for col in data.columns if not col.startswith('Unnamed:')]
data = data[filtered_columns]

data['LAT'] = pd.to_numeric(data['LAT'], errors='coerce')
data['LONG'] = pd.to_numeric(data['LONG'], errors='coerce')
data['Stop Seq'] = pd.to_numeric(data['Stop Seq'], errors='coerce')
df=data.copy()

In [48]:
print(df.shape)
df = df.dropna(subset=['LAT', 'LONG'], how='all')
print(df.shape)

(34089, 9)
(34084, 9)


In [49]:
median_latitude = df['LAT'].median()
median_longitude = df['LONG'].median()
print(median_latitude,median_longitude)

18.5250397 73.8452698


In [50]:
def determine_zone(lat, lon, median_lat, median_lon):
    if lat < median_lat:
        if lon < median_lon:
            return 'Zone 1'
        else:
            return 'Zone 2'
    else:
        if lon < median_lon:
            return 'Zone 4'
        else:
            return 'Zone 3'

df['Zone'] = df.apply(lambda row: determine_zone(row['LAT'], row['LONG'], median_latitude, median_longitude), axis=1)

In [51]:
df['Capacity'] = 50
df['Speed range'] = 51
df['Direction'] = df['Route'].astype(str).apply(
    lambda x: 'Down' if x.endswith('D') else 
              'Up' if x.endswith('U') else 
              'Right' if x.endswith('R') else 'Unknown'
)
df['Coordinates'] = list(zip(df['LAT'], df['LONG']))
df.sort_values(by=['Route', 'Stop Seq'], inplace=True)

buses_data = []

for route, group in df.groupby('Route'):
    BusStopArray = group['Stop Code'].tolist()
    Source_stop = group.loc[group['Stop Seq'].idxmin(), 'Stop Code']
    Destination_Stop = group.loc[group['Stop Seq'].idxmax(), 'Stop Code']
    Direction = group['Direction'].iloc[0]
    Coordinates = group['Coordinates'].tolist()  

    buses_data.append({
        'BusID': len(buses_data) + 1,
        'RouteAccess': route,
        'BusStopArray': BusStopArray,
        'Capacity': 50,
        'Speed range': 51,
        'Direction': Direction,
        'Source stop': Source_stop,
        'Destination Stop': Destination_Stop,
        'Coordinates': Coordinates
    })

buses = pd.DataFrame(buses_data)

In [52]:
buses[buses['Direction']=='Unknown']

,BusID,RouteAccess,BusStopArray,Capacity,Speed range,Direction,Source stop,Destination Stop,Coordinates


In [53]:
buses['Zone']=df['Zone']

In [54]:
import pandas as pd
updated_buses = buses.copy()
updated_buses.to_csv('Buses_Data_Cleaned.csv')

In [55]:
buses=pd.read_csv('Buses_Data_Cleaned.csv')

In [56]:
import pandas as pd
from math import radians, sin, cos, sqrt, atan2
import re

def haversine(lon1, lat1, lon2, lat2):
    """Calculate the great circle distance between two points on the earth (specified in decimal degrees)"""
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    distance = 6371 * c  # Earth radius in kilometers
    return distance

def parse_coordinates(coord_str):
    """Parse the string representation of coordinates into a list of tuples"""
    coord_pairs = re.findall(r'\(([^,]+), ([^)]+)\)', coord_str)
    return [(float(lat), float(lon)) for lat, lon in coord_pairs]

def calculate_total_length(coords_str):
    """Calculate the total length of the route given a string of coordinates"""
    coords = parse_coordinates(coords_str)
    total_distance = 0
    for i in range(1, len(coords)):
        lon1, lat1 = coords[i-1]
        lon2, lat2 = coords[i]
        total_distance += haversine(lon1, lat1, lon2, lat2)
    return total_distance

In [57]:
buses['Total Length'] = buses['Coordinates'].apply(calculate_total_length)

In [58]:
nan_rows = buses[buses['Total Length'].isna()]
print(nan_rows[['Coordinates']])


Empty DataFrame
Columns: [Coordinates]
Index: []


In [59]:
buses[buses['Total Length'].isna()]

,Unnamed: 0,BusID,RouteAccess,BusStopArray,Capacity,Speed range,Direction,Source stop,Destination Stop,Coordinates,Zone,Total Length


In [60]:
df=buses.copy()

In [61]:
df = df.drop(df.columns[[0]], axis=1)

In [62]:

from ast import literal_eval

def calculate_stops(bus_stop_array_str):
    bus_stop_array = literal_eval(bus_stop_array_str)
    return len(bus_stop_array)

df['Number of Stops'] = df['BusStopArray'].apply(calculate_stops)

min_stops = df['Number of Stops'].min()
max_stops = df['Number of Stops'].max()


k = (10000 - 1000) / (max_stops - min_stops) if max_stops != min_stops else 0

df['Construction Cost'] = df['Number of Stops'].apply(lambda x: 10000 + k * (x - min_stops))

df[['Number of Stops', 'Construction Cost']].head()


,Number of Stops,Construction Cost
0,30,12853.658537
1,27,12524.390244
2,38,13731.707317
3,38,13731.707317
4,44,14390.243902


In [63]:
min_length = df['Total Length'].min()
max_length = df['Total Length'].max()
k_service = (5000 - 500) / (max_length - min_length) if max_length != min_length else 0
df['Service Cost'] = df['Total Length'].apply(lambda x: 5000 + k_service * (x - min_length))

In [64]:
df

,BusID,RouteAccess,BusStopArray,Capacity,Speed range,Direction,Source stop,Destination Stop,Coordinates,Zone,Total Length,Number of Stops,Construction Cost,Service Cost
0,1,10-D,"['10-D-00', '10-D-01', '10-D-02', '10-D-03', '...",50,51,Down,10-D-00,10-D-29,"[(18.5348383, 73.944175), (18.5335083, 73.9408...",Zone 4,10.486049,30,12853.658537,5527.946207
1,2,10-U,"['10-U-00', '10-U-01', '10-U-02', '10-U-03', '...",50,51,Up,10-U-00,10-U-26,"[(18.4993533, 73.8581423), (18.5005347, 73.859...",Zone 4,10.233062,27,12524.390244,5513.974700
2,3,100-D,"['100-D-01', '100-D-02', '100-D-03', '100-D-04...",50,51,Down,100-D-01,100-D-38,"[(18.576553, 73.688178), (18.5810162, 73.68770...",Zone 4,19.979368,38,13731.707317,6052.225783
3,4,100-U,"['100-U-01', '100-U-02', '100-U-03', '100-U-04...",50,51,Up,100-U-01,100-U-38,"[(18.52357, 73.85488), (18.5263090000003, 73.8...",Zone 4,19.314590,38,13731.707317,6015.512644
4,5,101-D,"['101-D-00', '101-D-01', '101-D-02', '101-D-03...",50,51,Down,101-D-00,101-D-43,"[(18.45419, 73.89066), (18.4573536, 73.8908234...",Zone 4,13.352752,44,14390.243902,5686.263186
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
956,957,VJR4-U,"['VJR4-U-01', 'VJR4-U-02', 'VJR4-U-03', 'VJR4-...",50,51,Up,VJR4-U-01,VJR4-U-33,"[(18.507995, 73.7956883), (18.507865, 73.79711...",Zone 1,11.845330,33,13182.926829,5603.014056
957,958,VJR5-D,"['VJR5-D-01', 'VJR5-D-02', 'VJR5-D-03', 'VJR5-...",50,51,Down,VJR5-D-01,VJR5-D-33,"[(18.6606594, 73.7319836), (18.6535249999998, ...",Zone 1,16.746208,33,13182.926829,5873.670736
958,959,VJR5-U,"['VJR5-U-01', 'VJR5-U-02', 'VJR5-U-03', 'VJR5-...",50,51,Up,VJR5-U-01,VJR5-U-33,"[(18.52357, 73.85488), (18.5263090000003, 73.8...",Zone 1,16.312106,33,13182.926829,5849.696965
959,960,VJR6-D,"['VJR6-D-01', 'VJR6-D-02', 'VJR6-D-03', 'VJR6-...",50,51,Down,VJR6-D-01,VJR6-D-38,"[(18.576553, 73.688178), (18.5810162, 73.68770...",Zone 1,18.837938,38,13731.707317,5989.188959


In [65]:
import pandas as pd
import numpy as np

def generate_checkin_checkout(num_stops):
    checkin_list = [np.random.randint(1, 70)] 
    checkout_list = [0]
    passengers_on_bus = checkin_list[0] - checkout_list[0] 

    for _ in range(1, num_stops):
        next_checkin = np.random.randint(1, 70)
        max_checkout = passengers_on_bus + next_checkin - 70
        next_checkout = max(0, min(passengers_on_bus, max_checkout))
        checkin_list.append(next_checkin)
        checkout_list.append(next_checkout)
        passengers_on_bus += next_checkin - next_checkout

    return checkin_list, checkout_list


np.random.seed(42)
df['CheckIn1'], df['CheckOut1'] = zip(*df['Number of Stops'].apply(generate_checkin_checkout))

df['CheckIn1'] = df['CheckIn1'].apply(list)
df['CheckOut1'] = df['CheckOut1'].apply(list)

for index, row in df.iterrows():
    if any((ci - co) > 70 for ci, co in zip(np.cumsum(row['CheckIn1']), np.cumsum(row['CheckOut1']))):
        print(f"Row {index} violates the condition for 1st.")
        break
        

np.random.seed(41)
df['CheckIn2'], df['CheckOut2'] = zip(*df['Number of Stops'].apply(generate_checkin_checkout))

df['CheckIn2'] = df['CheckIn2'].apply(list)
df['CheckOut2'] = df['CheckOut2'].apply(list)

for index, row in df.iterrows():
    if any((ci - co) > 70 for ci, co in zip(np.cumsum(row['CheckIn2']), np.cumsum(row['CheckOut2']))):
        print(f"Row {index} violates the condition for 2nd.")
        break


np.random.seed(40) 
df['CheckIn3'], df['CheckOut3'] = zip(*df['Number of Stops'].apply(generate_checkin_checkout))

df['CheckIn3'] = df['CheckIn3'].apply(list)
df['CheckOut3'] = df['CheckOut3'].apply(list)

for index, row in df.iterrows():
    if any((ci - co) > 70 for ci, co in zip(np.cumsum(row['CheckIn3']), np.cumsum(row['CheckOut3']))):
        print(f"Row {index} violates the condition for 3rd.")
        break


np.random.seed(39) 
df['CheckIn4'], df['CheckOut4'] = zip(*df['Number of Stops'].apply(generate_checkin_checkout))

df['CheckIn4'] = df['CheckIn4'].apply(list)
df['CheckOut4'] = df['CheckOut4'].apply(list)

for index, row in df.iterrows():
    if any((ci - co) > 70 for ci, co in zip(np.cumsum(row['CheckIn4']), np.cumsum(row['CheckOut4']))):
        print(f"Row {index} violates the condition for 4th.")
        break



np.random.seed(38) 
df['CheckIn5'], df['CheckOut5'] = zip(*df['Number of Stops'].apply(generate_checkin_checkout))

df['CheckIn5'] = df['CheckIn5'].apply(list)
df['CheckOut5'] = df['CheckOut5'].apply(list)

for index, row in df.iterrows():
    if any((ci - co) > 70 for ci, co in zip(np.cumsum(row['CheckIn5']), np.cumsum(row['CheckOut5']))):
        print(f"Row {index} violates the condition for 5th.")
        break


np.random.seed(37) 
df['CheckIn6'], df['CheckOut6'] = zip(*df['Number of Stops'].apply(generate_checkin_checkout))

df['CheckIn6'] = df['CheckIn6'].apply(list)
df['CheckOut6'] = df['CheckOut6'].apply(list)

for index, row in df.iterrows():
    if any((ci - co) > 70 for ci, co in zip(np.cumsum(row['CheckIn6']), np.cumsum(row['CheckOut6']))):
        print(f"Row {index} violates the condition for 6th.")
        break
        
  
np.random.seed(36) 
df['CheckIn7'], df['CheckOut7'] = zip(*df['Number of Stops'].apply(generate_checkin_checkout))

df['CheckIn7'] = df['CheckIn7'].apply(list)
df['CheckOut7'] = df['CheckOut7'].apply(list)

for index, row in df.iterrows():
    if any((ci - co) > 70 for ci, co in zip(np.cumsum(row['CheckIn7']), np.cumsum(row['CheckOut7']))):
        print(f"Row {index} violates the condition for 7th.")
        break
        
          
np.random.seed(35)  
df['CheckIn8'], df['CheckOut8'] = zip(*df['Number of Stops'].apply(generate_checkin_checkout))

df['CheckIn8'] = df['CheckIn8'].apply(list)
df['CheckOut8'] = df['CheckOut8'].apply(list)

for index, row in df.iterrows():
    if any((ci - co) > 70 for ci, co in zip(np.cumsum(row['CheckIn8']), np.cumsum(row['CheckOut8']))):
        print(f"Row {index} violates the condition for 8th.")
        break
        

  
np.random.seed(34) 
df['CheckIn9'], df['CheckOut9'] = zip(*df['Number of Stops'].apply(generate_checkin_checkout))

df['CheckIn9'] = df['CheckIn9'].apply(list)
df['CheckOut9'] = df['CheckOut9'].apply(list)

for index, row in df.iterrows():
    if any((ci - co) > 70 for ci, co in zip(np.cumsum(row['CheckIn9']), np.cumsum(row['CheckOut9']))):
        print(f"Row {index} violates the condition for 9th.")
        break
        
           
np.random.seed(33) 
df['CheckIn10'], df['CheckOut10'] = zip(*df['Number of Stops'].apply(generate_checkin_checkout))

df['CheckIn10'] = df['CheckIn10'].apply(list)
df['CheckOut10'] = df['CheckOut10'].apply(list)

for index, row in df.iterrows():
    if any((ci - co) > 70 for ci, co in zip(np.cumsum(row['CheckIn10']), np.cumsum(row['CheckOut10']))):
        print(f"Row {index} violates the condition for 10th.")
        break
        
        
 
np.random.seed(32) 
df['CheckIn11'], df['CheckOut11'] = zip(*df['Number of Stops'].apply(generate_checkin_checkout))


df['CheckIn11'] = df['CheckIn11'].apply(list)
df['CheckOut11'] = df['CheckOut11'].apply(list)


for index, row in df.iterrows():
    if any((ci - co) > 70 for ci, co in zip(np.cumsum(row['CheckIn11']), np.cumsum(row['CheckOut11']))):
        print(f"Row {index} violates the condition for 11th.")
        break
        
           
np.random.seed(31)  
df['CheckIn12'], df['CheckOut12'] = zip(*df['Number of Stops'].apply(generate_checkin_checkout))

df['CheckIn12'] = df['CheckIn12'].apply(list)
df['CheckOut12'] = df['CheckOut12'].apply(list)

for index, row in df.iterrows():
    if any((ci - co) > 70 for ci, co in zip(np.cumsum(row['CheckIn12']), np.cumsum(row['CheckOut12']))):
        print(f"Row {index} violates the condition for 12th.")
        break




In [66]:
print(df.columns.tolist())


['BusID', 'RouteAccess', 'BusStopArray', 'Capacity', 'Speed range', 'Direction', 'Source stop', 'Destination Stop', 'Coordinates', 'Zone', 'Total Length', 'Number of Stops', 'Construction Cost', 'Service Cost', 'CheckIn1', 'CheckOut1', 'CheckIn2', 'CheckOut2', 'CheckIn3', 'CheckOut3', 'CheckIn4', 'CheckOut4', 'CheckIn5', 'CheckOut5', 'CheckIn6', 'CheckOut6', 'CheckIn7', 'CheckOut7', 'CheckIn8', 'CheckOut8', 'CheckIn9', 'CheckOut9', 'CheckIn10', 'CheckOut10', 'CheckIn11', 'CheckOut11', 'CheckIn12', 'CheckOut12']


In [67]:
import pandas as pd
import ast 

def parse_coordinates(coord_string):
    return ast.literal_eval(coord_string.replace('),', '),'))

def create_stop_coordinate_mapping(row):
    stops = ast.literal_eval(row['BusStopArray'])
    coordinates = parse_coordinates(row['Coordinates'])
    return [{stop: {'lat': coord[0], 'long': coord[1]}} for stop, coord in zip(stops, coordinates)]

df['StopCoordinateMap'] = df.apply(create_stop_coordinate_mapping, axis=1)

print(df['StopCoordinateMap'].head())


0    [{'10-D-00': {'lat': 18.5348383, 'long': 73.94...
1    [{'10-U-00': {'lat': 18.4993533, 'long': 73.85...
2    [{'100-D-01': {'lat': 18.576553, 'long': 73.68...
3    [{'100-U-01': {'lat': 18.52357, 'long': 73.854...
4    [{'101-D-00': {'lat': 18.45419, 'long': 73.890...
Name: StopCoordinateMap, dtype: object


In [68]:
print(df.columns.tolist())

['BusID', 'RouteAccess', 'BusStopArray', 'Capacity', 'Speed range', 'Direction', 'Source stop', 'Destination Stop', 'Coordinates', 'Zone', 'Total Length', 'Number of Stops', 'Construction Cost', 'Service Cost', 'CheckIn1', 'CheckOut1', 'CheckIn2', 'CheckOut2', 'CheckIn3', 'CheckOut3', 'CheckIn4', 'CheckOut4', 'CheckIn5', 'CheckOut5', 'CheckIn6', 'CheckOut6', 'CheckIn7', 'CheckOut7', 'CheckIn8', 'CheckOut8', 'CheckIn9', 'CheckOut9', 'CheckIn10', 'CheckOut10', 'CheckIn11', 'CheckOut11', 'CheckIn12', 'CheckOut12', 'StopCoordinateMap']


In [69]:

df = df.rename(columns={
    'BusID': 'BusId',
    'BusStopArray': 'BusStops',
    'Speed range': 'Speed',
    'Source stop': 'Source',
    'Destination Stop': 'Destination',    
})


In [70]:
df

,BusId,RouteAccess,BusStops,Capacity,Speed,Direction,Source,Destination,Coordinates,Zone,...,CheckOut8,CheckIn9,CheckOut9,CheckIn10,CheckOut10,CheckIn11,CheckOut11,CheckIn12,CheckOut12,StopCoordinateMap
0,1,10-D,"['10-D-00', '10-D-01', '10-D-02', '10-D-03', '...",50,51,Down,10-D-00,10-D-29,"[(18.5348383, 73.944175), (18.5335083, 73.9408...",Zone 4,...,"[0, 2, 34, 64, 65, 12, 12, 57, 58, 56, 45, 56,...","[34, 69, 36, 12, 30, 19, 30, 30, 21, 63, 6, 21...","[0, 33, 36, 12, 30, 19, 30, 30, 21, 63, 6, 21,...","[21, 8, 67, 19, 58, 68, 62, 15, 43, 46, 36, 34...","[0, 0, 26, 19, 58, 68, 62, 15, 43, 46, 36, 34,...","[44, 6, 55, 63, 20, 4, 10, 5, 12, 4, 66, 35, 3...","[0, 0, 35, 63, 20, 4, 10, 5, 12, 4, 66, 35, 36...","[17, 24, 59, 29, 43, 47, 7, 19, 26, 24, 29, 11...","[0, 0, 30, 29, 43, 47, 7, 19, 26, 24, 29, 11, ...","[{'10-D-00': {'lat': 18.5348383, 'long': 73.94..."
1,2,10-U,"['10-U-00', '10-U-01', '10-U-02', '10-U-03', '...",50,51,Up,10-U-00,10-U-26,"[(18.4993533, 73.8581423), (18.5005347, 73.859...",Zone 4,...,"[0, 0, 41, 4, 63, 17, 46, 53, 34, 68, 30, 45, ...","[63, 58, 45, 20, 39, 24, 10, 14, 9, 37, 49, 14...","[0, 51, 45, 20, 39, 24, 10, 14, 9, 37, 49, 14,...","[26, 31, 54, 56, 61, 2, 63, 50, 37, 34, 48, 16...","[0, 0, 41, 56, 61, 2, 63, 50, 37, 34, 48, 16, ...","[51, 36, 62, 51, 46, 25, 12, 48, 44, 51, 69, 4...","[0, 17, 62, 51, 46, 25, 12, 48, 44, 51, 69, 49...","[33, 12, 66, 7, 36, 11, 29, 27, 27, 25, 52, 52...","[0, 0, 41, 7, 36, 11, 29, 27, 27, 25, 52, 52, ...","[{'10-U-00': {'lat': 18.4993533, 'long': 73.85..."
2,3,100-D,"['100-D-01', '100-D-02', '100-D-03', '100-D-04...",50,51,Down,100-D-01,100-D-38,"[(18.576553, 73.688178), (18.5810162, 73.68770...",Zone 4,...,"[0, 0, 30, 63, 55, 29, 61, 49, 6, 45, 6, 37, 6...","[7, 60, 30, 64, 18, 1, 44, 45, 50, 39, 24, 16,...","[0, 0, 27, 64, 18, 1, 44, 45, 50, 39, 24, 16, ...","[60, 60, 20, 44, 33, 61, 49, 29, 61, 52, 5, 42...","[0, 50, 20, 44, 33, 61, 49, 29, 61, 52, 5, 42,...","[41, 18, 10, 33, 69, 1, 65, 67, 38, 9, 60, 61,...","[0, 0, 0, 32, 69, 1, 65, 67, 38, 9, 60, 61, 63...","[6, 61, 23, 7, 62, 21, 28, 45, 28, 35, 31, 32,...","[0, 0, 20, 7, 62, 21, 28, 45, 28, 35, 31, 32, ...","[{'100-D-01': {'lat': 18.576553, 'long': 73.68..."
3,4,100-U,"['100-U-01', '100-U-02', '100-U-03', '100-U-04...",50,51,Up,100-U-01,100-U-38,"[(18.52357, 73.85488), (18.5263090000003, 73.8...",Zone 4,...,"[0, 19, 33, 24, 33, 55, 49, 25, 5, 62, 20, 43,...","[44, 38, 57, 63, 34, 52, 13, 57, 10, 69, 42, 6...","[0, 12, 57, 63, 34, 52, 13, 57, 10, 69, 42, 63...","[61, 20, 22, 65, 13, 61, 19, 66, 6, 60, 46, 19...","[0, 11, 22, 65, 13, 61, 19, 66, 6, 60, 46, 19,...","[1, 17, 64, 14, 13, 68, 38, 48, 69, 6, 10, 11,...","[0, 0, 12, 14, 13, 68, 38, 48, 69, 6, 10, 11, ...","[17, 36, 1, 15, 24, 51, 6, 65, 67, 36, 30, 41,...","[0, 0, 0, 0, 23, 51, 6, 65, 67, 36, 30, 41, 48...","[{'100-U-01': {'lat': 18.52357, 'long': 73.854..."
4,5,101-D,"['101-D-00', '101-D-01', '101-D-02', '101-D-03...",50,51,Down,101-D-00,101-D-43,"[(18.45419, 73.89066), (18.4573536, 73.8908234...",Zone 4,...,"[0, 0, 0, 6, 7, 48, 7, 50, 2, 22, 61, 10, 29, ...","[13, 10, 25, 19, 8, 56, 40, 6, 66, 25, 36, 24,...","[0, 0, 0, 0, 5, 56, 40, 6, 66, 25, 36, 24, 25,...","[3, 58, 25, 34, 8, 8, 17, 43, 29, 37, 61, 3, 2...","[0, 0, 16, 34, 8, 8, 17, 43, 29, 37, 61, 3, 21...","[27, 42, 23, 38, 25, 33, 35, 28, 44, 54, 63, 5...","[0, 0, 22, 38, 25, 33, 35, 28, 44, 54, 63, 5, ...","[67, 52, 7, 16, 26, 23, 3, 19, 37, 48, 26, 36,...","[0, 49, 7, 16, 26, 23, 3, 19, 37, 48, 26, 36, ...","[{'101-D-00': {'lat': 18.45419, 'long': 73.890..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
956,957,VJR4-U,"['VJR4-U-01', 'VJR4-U-02', 'VJR4-U-03', 'VJR4-...",50,51,Up,VJR4-U-01,VJR4-U-33,"[(18.507995, 73.7956883), (18.507865, 73.79711...",Zone 1,...,"[0, 55, 58, 14, 66, 19, 44, 11, 53, 13, 23, 17...","[29, 36, 66, 8, 42, 60, 64, 57, 46, 31, 28, 37...","[0, 0, 61, 8, 42, 60, 64, 57, 46, 31, 28, 37, ...","[18, 21, 63, 29, 24, 47, 38, 52, 

In [71]:
df.to_csv('BusRoutes.csv')